# Fonction tester

This file main goal is to test function and classes for later use in training.
Is ignore entirely during the training

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import keras

from tqdm import tqdm
from math import ceil
from generator import Diff_Generator, Basic_Generator
from preprocess import ProdKernel, FKernel, Preprocess, Zero_One,  Level_Normalizer, Normalizer
from utils import Load_FLX_dict, Plot_Batch

data_folder="Data"

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
##### Dictionnary
D = Load_FLX_dict()
##### Kernels
Klist = []
##### b_size
batch_size= 64
#nM.fit_generator(test_generator)

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, RNN, SimpleRNN, GRU, Activation, Flatten
from keras.layers import MaxPooling1D, Dense, Conv1D, UpSampling1D, AveragePooling1D
from keras.engine.input_layer import Input
from keras import optimizers
from keras.layers import Bidirectional
from keras import backend as K
from keras.losses import mean_squared_error

M = Sequential()
M.add(UpSampling1D(5))
M.add(AveragePooling1D(26, stride=5, padding='same', name='AVG1'))
#M.add(Dense(1))
#M.add(Flatten())
#M.summary()
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1.e-5)

from keras.models import Model

newInput = Input(shape=(72,11))
newOutputs = M(newInput)
nM = Model(newInput, newOutputs)
nM.compile(loss='mse', optimizer=rmsprop)
nM.summary()
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 72, 11)            0         
_________________________________________________________________
Seq (Sequential)             (None, 72, 11)            0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling1d_8 (UpSampling1 (None, 360, 11)           0         
_________________________________________________________________
AVG1 (AveragePooling1D)      (None, 72, 11)            0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(26, padding="same", name="AVG1", strides=5)`
  if sys.path[0] == '':


In [4]:
import matplotlib.pyplot as plt

def multipred(models,x):
    bs = x.shape[0]
    yp = np.zeros((len(models),bs,73))
    for i,model in enumerate(models):
        yp[i,:,:]=model.predict(x)
    return(yp)

testsize = 128*4
for x0,y0 in test_generator( (x,y), batch_size=testsize, nb=1):
    data_test = (x0,y0)

ytruth = data_test[1]
xtest = data_test[0].copy() 
xtest1 = data_test[0].copy() 
xtest2 = data_test[0].copy()
xtest3 = data_test[0].copy()

xtest1 *=0
xtest2[:,:,1] *= 0
xtest3[:,:,2] *= 0

ypred = multipred(models,xtest)
ypred1 = multipred(models,xtest1)
ypred2 = multipred(models,xtest2)
ypred3 = multipred(models,xtest3)

def double_mean_squared(y_true, y_pred):
    fist_loss   =  K.mean(K.square(y_pred - y_true), axis=-1)
    second_loss =  x[:, :1]
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
return K.mean(K.square(first_log - second_log), axis=-1)

